# Prática Guiada: Maldição da Dimensionalidade

In [0]:
%pylab inline
import seaborn as sns
import pandas as pd

## Proporção de outliers

Se temos uma variável distribuída uniformemente em um hipercubo de $d$ dimensões, qual proporção de outliers encontramos? 

Podemos definir os outliers como aqueles pontos que recebem valores extremos em algumas das $d$ dimensões. 

Por exemplo, se pensamos em duas dimensões, forma-se um quadrado.

**Obs.:** Este código serve para gerar um conjunto de dados artificial. Não é fundamental entendê-lo em profundidade.

In [0]:
m = 500
xs = uniform(0.0,1.0,m)
ys = uniform(0.0,1.0,m)
X = c_[xs,ys]
plot(xs,ys,'o',ms=2);
x = [0.0,1.0]
p = 0.01 # 1%
fill_between(x,0,p,alpha=0.2,color='k')
fill_between(x,1-p,1,alpha=0.2,color='k')
fill_betweenx(x,0,p,alpha=0.2,color='k')
fill_betweenx(x,1-p,1,alpha=0.2,color='k',label='Região de outliers');
legend();
print('Outliers ',sum([any([(d &lt; .01 or d > .99) for d in p]) for p in X]),'de',m)

Mas o que acontece à medida que aumentamos a dimensionalidade do conjunto de dados? Como não podemos fazer gráficos com mais de três dimensões, o que vamos fazer é um gráfico da evolução da proporção de outliers para cada nível de dimensionalidade.

In [0]:
def sample_(d, N=1000):
    ‘‘‘Gera uma amostra de 1000 pontos em d dimensões’’’
    return [[uniform(0., 1.) for i in range(d)] for _ in range(N)]

def corner_count(points):
    '''Conta a quantidade de pontos que são outliers em alguma das d dimensões.'''
    return mean([any([(d &lt; .01 or d > .99) for d in p]) for p in points])

Ds=arange(1,200)
plot(Ds, array([corner_count(sample_(d)) for d in Ds])*100);
xlabel('Proporção')
ylabel('Dimensões');

* O que vocês observam?
* O que acontece com a proporção de outliers à medida que o espaço de previsores aumenta em dimensionalidade?
* Qual pode ser a causa disso?

## Vamos retomar o exemplo de regressão logística

A seguir, vamos ver o que acontece com a performance de uma regressão logística sem regularização à medida que a quantidade de dimensões aumenta.

A regressão deverá prever dados simulados por meio de distribuições normais multivariadas.

* A classe 0 terá média 0 e as variâncias de todas as dimensões valerão 0,5
* A classe 1 terá média 1 e as variâncias de todas as dimensões valerão 0,5

### Regressão logística sobre dados de diferentes dimensões

Geramos una lista de possíveis dimensões e para cada um desses valores geramos um conjunto de dados simulado com as características descritas acima.

Sobre esses dados rodamos e avaliamos uma regressão logística para cada dimensionalidade. A métrica de avaliação será a média do score de teste, utilizando cross validation com uma partição de 5 folds em todos os casos. 

In [0]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
score = []
# Avaliamos o modelo em todas essas dimensões
ds = [1,3,6,40,50,60,120,180]

for d in ds:

    size0 = 40; size1 = 40;
    
    loc0 = ones(d); loc1 = zeros(d)
    
    sigma0 = 0.5; sigma1 = 0.5; C0 = sigma0*diag(ones(d)); C1 = sigma1*diag(ones(d))

    #Geramos dados multivariados
    x = c_[multivariate_normal(loc0,C0,size0),zeros(size0)]
    y = c_[multivariate_normal(loc1,C1,size1),ones(size1)]
    X = r_[x,y]

    # Criamos a matriz de features
    dfX = pd.DataFrame(X)
    dfX.columns = ['dim'+str(i) for i in range(d)]+['class']
    X = dfX.drop('class',axis=1)
    y = dfX['class']
    
    # Salvamos a média do score de cross-validation
    model = LogisticRegression(C = 1e10,n_jobs=4)
    kf = KFold(5, shuffle=True, random_state=0)
    score.append(np.mean(cross_val_score(model,X,y,cv=kf)))

Observamos que a partir de certa quantidade de dimensões, a performance começa a cair muito por causa do excesso de dimensões em relação com a quantidade de dados disponíveis

In [0]:
score

In [0]:
plt.plot(ds,score,'o-');
ylabel('Accuracy')
xlabel('Dimensions');

Se o volume de dados aumentar, a quantidade ótima de dimensões também vai aumentar. 

In [0]:
score = []
# Avaliamos o modelo em todas essas dimensões
ds = [1,3,6,40,50,60,120,180]

for d in ds:

    size0 = 4000; size1 = 4000;
    
    loc0 = ones(d); loc1 = zeros(d)
    
    sigma0 = 0.5; sigma1 = 0.5; C0 = sigma0*diag(ones(d)); C1 = sigma1*diag(ones(d))

    #Geramos dados multivariados
    x = c_[multivariate_normal(loc0,C0,size0),zeros(size0)]
    y = c_[multivariate_normal(loc1,C1,size1),ones(size1)]
    X = r_[x,y]

    # Criamos a matriz de features
    dfX = pd.DataFrame(X)
    dfX.columns = ['dim'+str(i) for i in range(d)]+['class']
    X = dfX.drop('class',axis=1)
    y = dfX['class']
    
    # Salvamos a média do score de cross-validation
    model = LogisticRegression(C = 1e10,n_jobs=4)
    kf = KFold(5, shuffle=True, random_state=0)
    score.append(np.mean(cross_val_score(model,X,y,cv=kf)))

In [0]:
plt.plot(ds,score,'o-');
ylabel('Accuracy')
xlabel('Dimensions');